In [27]:
import pandas as pd # dataframes
import numpy as np # algebra & calculus
import nltk # text preprocessing & manipulation
# from textblob import TextBlob
import matplotlib.pyplot as plt # plotting
import seaborn as sns # plotting
%matplotlib inline
from functools import partial # to reduce df memory consumption by applying to_numeric

color = sns.color_palette() # adjusting plotting style
import warnings
warnings.filterwarnings('ignore') # silence annoying warnings

In [28]:
## count data-set for each csv file
# aisles
aisles = pd.read_csv('aisles.csv', engine='c')
print('Total aisles: {}'.format(aisles.shape[0]))

# departments
departments = pd.read_csv('departments.csv', engine='c')
print('Total departments: {}'.format(departments.shape[0]))

# products
products = pd.read_csv('products.csv', engine='c')
print('Total products: {}'.format(products.shape[0]))

Total aisles: 134
Total departments: 21
Total products: 49688


In [29]:
aisles

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation
5,6,other
6,7,packaged meat
7,8,bakery desserts
8,9,pasta sauce
9,10,kitchen supplies


In [30]:
departments

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol
5,6,international
6,7,beverages
7,8,pets
8,9,dry goods pasta
9,10,bulk


In [31]:
products.tail(3)

,product_id,product_name,aisle_id,department_id
49685,49686,Artisan Baguette,112,3
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8
49687,49688,Fresh Foaming Cleanser,73,11


In [32]:
# combine aisles, departments and products (left joined to products)
goods = pd.merge(left=pd.merge(left=products, right=departments, how='left'), right=aisles, how='left')
goods.tail()

,product_id,product_name,aisle_id,department_id,department,aisle
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5,alcohol,spirits
49684,49685,En Croute Roast Hazelnut Cranberry,42,1,frozen,frozen vegan vegetarian
49685,49686,Artisan Baguette,112,3,bakery,bread
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8,pets,cat food care
49687,49688,Fresh Foaming Cleanser,73,11,personal care,facial care


In [33]:
# to retain '-' and make product names more "standard"
goods.product_name = goods.product_name.str.replace(' ', '_').str.lower() 
goods.product_name.tail()

49683     vodka,_triple_distilled,_twist_of_vanilla
49684            en_croute_roast_hazelnut_cranberry
49685                              artisan_baguette
49686    smartblend_healthy_metabolism_dry_cat_food
49687                        fresh_foaming_cleanser
Name: product_name, dtype: object

In [34]:
# retype goods to reduce memory usage
goods.product_id = goods.product_id.astype(np.int32)
goods.aisle_id = goods.aisle_id.astype(np.int16)
goods.department_id = goods.department_id.astype(np.int8)

In [35]:
## load datasets
# train dataset
op_train = pd.read_csv('order_products__train.csv', engine='c', 
                       dtype={'order_id': np.int32, 
                              'product_id': np.int32, 
                              'add_to_cart_order': np.int16, 
                              'reordered': np.int8})
print('Total ordered products(train): {}'.format(op_train.shape[0]))

Total ordered products(train): 1384617


In [36]:
op_train.tail(3)

,order_id,product_id,add_to_cart_order,reordered
1384614,3421070,35951,1,1
1384615,3421070,16953,2,1
1384616,3421070,4724,3,1


In [37]:
# test dataset (submission)
test = pd.read_csv('sample_submission.csv', engine='c')
print('Total orders(test): {}'.format(test.shape[0]))

Total orders(test): 75000


In [38]:
test

,order_id,products
0,17,39276 29259
1,34,39276 29259
2,137,39276 29259
3,182,39276 29259
4,257,39276 29259
5,313,39276 29259
6,353,39276 29259
7,386,39276 29259
8,414,39276 29259
9,418,39276 29259


In [39]:
#prior dataset
op_prior = pd.read_csv('order_products__prior.csv', engine='c', 
                       dtype={'order_id': np.int32, 
                              'product_id': np.int32, 
                              'add_to_cart_order': np.int16, 
                              'reordered': np.int8})

print('Total ordered products(prior): {}'.format(op_prior.shape[0]))

Total ordered products(prior): 32434489


In [40]:
op_prior

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0
5,2,17794,6,1
6,2,40141,7,1
7,2,1819,8,1
8,2,43668,9,0
9,3,33754,1,1


In [41]:
# orders
orders = pd.read_csv('orders.csv', engine='c', dtype={'order_id': np.int32, 
                                                      'user_id': np.int32, 
                                                      'order_number': np.int16,  # max 100, could use int8
                                                      'order_dow': np.int8, 
                                                      'order_hour_of_day': np.int8, 
                                                      'days_since_prior_order': np.float16})
print('Total orders: {}'.format(orders.shape[0]))

Total orders: 3421083


In [42]:
orders

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
5,3367565,1,prior,6,2,7,19.0
6,550135,1,prior,7,1,9,20.0
7,3108588,1,prior,8,1,14,14.0
8,2295261,1,prior,9,1,16,0.0
9,2550362,1,prior,10,4,8,30.0


In [43]:
# numerization of eval_set 
orders.eval_set = orders.eval_set.replace({'prior': 0, 'train': 1, 'test':2}).astype(np.int8)
orders.days_since_prior_order = orders.days_since_prior_order.fillna(-1).astype(np.int8) # int형으로 변환, 누락된 값 -1로 치환

In [44]:
orders

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,0,1,2,8,-1
1,2398795,1,0,2,3,7,15
2,473747,1,0,3,3,12,21
3,2254736,1,0,4,4,7,29
4,431534,1,0,5,4,15,28
5,3367565,1,0,6,2,7,19
6,550135,1,0,7,1,9,20
7,3108588,1,0,8,1,14,14
8,2295261,1,0,9,1,16,0
9,2550362,1,0,10,4,8,30


In [45]:
from functools import partial

# merge train and prior together iteratively, to fit into 8GB kernel RAM
# split df indexes into parts ()
indexes = np.linspace(0, len(op_prior), num=10, dtype=np.int32)

# initialize it with train dataset
train_details = pd.merge(
                left=op_train,
                 right=orders, 
                 how='left', 
                 on='order_id'
        ).apply(partial(pd.to_numeric, errors='ignore', downcast='integer'))

# add order hierarchy
train_details = pd.merge(
                left=train_details,
                right=goods[['product_id', 
                             'aisle_id', 
                             'department_id']].apply(partial(pd.to_numeric, 
                                                             errors='ignore', 
                                                             downcast='integer')),
                how='left',
                on='product_id'
)

print(train_details.shape, op_train.shape)

# delete (redundant now) dataframes
#del op_train

#order_details.head()

(1384617, 12) (1384617, 4)


In [46]:
indexes

array([       0,  3603832,  7207664, 10811496, 14415328, 18019160,
       21622992, 25226824, 28830656, 32434489])

In [47]:
train_details

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,aisle_id,department_id
0,1,49302,1,1,112108,1,4,4,10,9,120,16
1,1,11109,2,1,112108,1,4,4,10,9,108,16
2,1,10246,3,0,112108,1,4,4,10,9,83,4
3,1,49683,4,0,112108,1,4,4,10,9,83,4
4,1,43633,5,1,112108,1,4,4,10,9,95,15
5,1,13176,6,0,112108,1,4,4,10,9,24,4
6,1,47209,7,0,112108,1,4,4,10,9,24,4
7,1,22035,8,1,112108,1,4,4,10,9,21,16
8,36,39612,1,0,79431,1,23,6,18,30,2,16
9,36,19660,2,1,79431,1,23,6,18,30,115,7


In [48]:
%%time
# update by small portions
order_details = pd.merge(left=pd.merge(
                         left=op_prior,
                         right=goods[['product_id', 
                                      'aisle_id', 
                                      'department_id' ]],
                         how='left',
                         on='product_id'
                         ),
                         right=orders, 
                         how='left', 
                         on='order_id')
        
print('Datafame length: {}'.format(order_details.shape[0]))
print('Memory consumption: {:.2f} Mb'.format(sum(order_details.memory_usage(index=True, 
                                                                         deep=True) / 2**20)))
# check dtypes to see if we use memory effectively
#print(order_details.dtypes)

# make sure we didn't forget to retain test dataset :D
#test_orders = orders[orders.eval_set == 2]

Datafame length: 32434489
Memory consumption: 989.82 Mb
Wall time: 14.2 s


In [49]:
order_details

,order_id,product_id,add_to_cart_order,reordered,aisle_id,department_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2,33120,1,1,86,16,202279,0,3,5,9,8
1,2,28985,2,1,83,4,202279,0,3,5,9,8
2,2,9327,3,0,104,13,202279,0,3,5,9,8
3,2,45918,4,1,19,13,202279,0,3,5,9,8
4,2,30035,5,0,17,13,202279,0,3,5,9,8
5,2,17794,6,1,83,4,202279,0,3,5,9,8
6,2,40141,7,1,105,13,202279,0,3,5,9,8
7,2,1819,8,1,88,13,202279,0,3,5,9,8
8,2,43668,9,0,123,4,202279,0,3,5,9,8
9,3,33754,1,1,120,16,205970,0,16,5,17,12


In [50]:
train_orders = orders[orders.eval_set == 1]
train_orders

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
10,1187899,1,1,11,4,8,14
25,1492625,2,1,15,1,11,30
49,2196797,5,1,5,0,11,6
74,525192,7,1,21,2,11,6
78,880375,8,1,4,1,14,10
82,1094988,9,1,4,6,10,30
88,1822501,10,1,6,0,19,30
115,1827621,13,1,13,0,21,8
129,2316178,14,1,14,2,19,11
200,2180313,17,1,41,3,10,30


In [51]:
# switch to integer train indexes so .loc == .iloc
train_orders.index.name = 'raw_order'
train_orders.reset_index(inplace=True)
train_orders

,raw_order,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,10,1187899,1,1,11,4,8,14
1,25,1492625,2,1,15,1,11,30
2,49,2196797,5,1,5,0,11,6
3,74,525192,7,1,21,2,11,6
4,78,880375,8,1,4,1,14,10
5,82,1094988,9,1,4,6,10,30
6,88,1822501,10,1,6,0,19,30
7,115,1827621,13,1,13,0,21,8
8,129,2316178,14,1,14,2,19,11
9,200,2180313,17,1,41,3,10,30


In [52]:
import graphviz
import xgboost as xgb

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(order_details, test_size=0.25)

In [53]:
xgbr = xgb.XGBRegressor(n_estimators=100, max_depth=1)

In [54]:
xgbr.fit(X_train, y_train) 

NameError: name 'X_train' is not defined

In [45]:
def get_last_orders_reordered(test_orders):
    test_history = order_details[(order_details.user_id.isin(test_orders.user_id))]
    last_orders = test_history.groupby('user_id')['order_number'].max()
    
    t = pd.merge(
        left=pd.merge(
                left=last_orders.reset_index(),
                right=test_history[test_history.reordered == 1],
                how='left',
                on=['user_id', 'order_number']
            )[['user_id', 'product_id']],
        right=test_orders[['user_id', 'order_id']],
        how='left',
        on='user_id'
    ).fillna(-1).groupby('order_id')['product_id'].apply(lambda x: ' '.join([str(int(e)) for e in set(x)]) 
                                              ).reset_index().replace(to_replace='-1', value='None')
    t.columns = ['order_id', 'products']
    
    # occasionally there is a bug where a line with order_id == -1 makes it through. doesn't *seem* to effect things
    return t[t.order_id > 0].set_index('order_id')

Run the above function for 4 folds...  
Strictly speaking, this model does not have any interdependance on the train set, but to provide a complete demonstration KFold is used anyway.

In [46]:
import sklearn.model_selection

cvpreds = []

kf = sklearn.model_selection.KFold(4, shuffle=True, random_state=0)
for train_index, test_index in kf.split(train_orders.index):
    cvpreds.append(get_last_orders_reordered(train_orders.iloc[test_index]))

df_cvpreds = pd.concat(cvpreds).sort_index()
df_cvpreds.tail()

,products
order_id,
3421049,None
3421056,5750
3421058,21795 6244 36103 15629 24852 4347
3421063,22474 44717 19517 19057 9076 38293 30169 26714...
3421070,46149 2855


In [47]:
# Now to produce output (indentical to original notebook, so submission is not necessary!)
test_preds = get_last_orders_reordered(orders[orders.eval_set == 2])
test_preds.to_csv('cvtest-output.csv', encoding='utf-8')

### CV F1 validation code begins here  
Produce an equivalent .csv + DataFrame to output with the train ground truth data

In [48]:
try:
    df_train_gt = pd.read_csv('train.csv', index_col='order_id')
except:
    train_gtl = []

    for uid, subset in train_details.groupby('user_id'):
        subset1 = subset[subset.reordered == 1]
        oid = subset.order_id.values[0]

        if len(subset1) == 0:
            train_gtl.append((oid, 'None'))
            continue

        ostr = ' '.join([str(int(e)) for e in subset1.product_id.values])
        # .strip is needed because join can have a padding space at the end
        train_gtl.append((oid, ostr.strip()))

    df_train_gt = pd.DataFrame(train_gtl)

    df_train_gt.columns = ['order_id', 'products']
    df_train_gt.set_index('order_id', inplace=True)
    df_train_gt.sort_index(inplace=True)
    
    df_train_gt.to_csv('train.csv')

Now compare the ground truth and CV DataFrames

In [49]:
f1 = []
for gt, pred in zip(df_train_gt.sort_index().products, df_cvpreds.sort_index().products):
    lgt = gt.replace("None", "-1").split(' ')
    lpred = pred.replace("None", "-1").split(' ')
    
    rr = (np.intersect1d(lgt, lpred))
    precision = np.float(len(rr)) / len(lpred)
    recall = np.float(len(rr)) / len(lgt)

    denom = precision + recall
    f1.append(((2 * precision * recall) / denom) if denom > 0 else 0)

print(np.mean(f1))

0.32558651711
